# Aligning ESG Controversy Data with International Guidelines through Semi-Automatic Ontology Construction
Here is the notebook used for the research paper

In [1]:
import os
import zipfile
import pandas as pd
import json
import re
from pathlib import Path
from openai import AzureOpenAI
from tqdm import tqdm

### Dataset Creation
The data used in this paper comes from the public repository Webz.io https://github.com/Webhose/free-news-datasets

Since the data is separated into different categories, we preprocess the repository to have one big file with every single text documents in English, we are also interested in "Negative" news article, therefore we filter them too. 

In order to run this code, you need to clone or download the Webz.io reprository into a `data` folder

In [ ]:
base_folder = Path("data")
# Path to the folder containing the .zip files
folder_path = base_folder / "free-news-datasets-master" / "News_Datasets"
# Path to save the final concatenated DataFrame
output_path = base_folder / "research_paper_data.parquet"

In [ ]:
# Step 1: Iterate through all files in the folder and extract relevant zip files
for file_name in tqdm(os.listdir(folder_path), desc="Extracting zip files"):
    if file_name.endswith(".zip") and 'negative' in file_name:
        file_path = folder_path / file_name
        extract_path = folder_path / file_name[:-4]  # Remove '.zip' from the name
        extract_path.mkdir(exist_ok=True)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

# Step 2: Iterate over all extracted folders and process JSON files
webzio_data = pd.DataFrame()

iteration_count = 0
for root, dirs, files in os.walk(folder_path):
    iteration_count += 1

for root, dirs, files in tqdm(os.walk(folder_path), desc="Processing extracted folders", total=iteration_count):
    for sub_dir in dirs:
        sub_dir_path = Path(root) / sub_dir
        for file in os.listdir(sub_dir_path):
            if file.endswith('.json'):
                json_file_path = sub_dir_path / file
                try:
                    with open(json_file_path, 'r', encoding='utf-8') as json_file:
                        file_data = json.load(json_file)
                        df = pd.json_normalize(file_data)
                        # Filter for English articles only
                        df = df[df['language'] == 'english']
                        webzio_data = pd.concat([webzio_data, df], ignore_index=True)
                except Exception as e:
                    print(f"Error reading {json_file_path}: {e}")

print(f"Total documents: {len(webzio_data)}")

# Step 3: Save the final concatenated DataFrame to a parquet file
webzio_data = webzio_data.drop_duplicates(subset='uuid', keep='first')
webzio_data.reset_index(drop=True, inplace=True)
webzio_data.to_parquet(output_path, index=False)
print(f"Concatenated data saved to {output_path}")


### Dataset Filtering

Here we use Azure AI to call our LLM to filter out documents unrelated to ESG (according to RepRisk methodology)

In [223]:
MODEL_NAME = "gpt-4o-mini-research"
MODEL_NAME_STR = MODEL_NAME.replace("-", "_").replace(".", "_")
CHAT_COMPLETION_PARAMS = {
    "model": MODEL_NAME,
    "temperature": 0,
    "top_p": 0,
    "seed": 42,
}

client = AzureOpenAI(
    azure_endpoint=os.getenv('AZURE_ENDPOINT'),
    api_key=os.getenv('AZURE_API_KEY'),
    api_version=os.getenv('AZURE_API_VERSION')
)
MODEL_OUTPUT_PATH = base_folder / f"research_paper_data_{MODEL_NAME_STR}.parquet"

In [224]:
webzio_data['ai_response'] = 0

for index, row in tqdm(webzio_data.iterrows(), total=len(webzio_data)):
    title = str(row.get('title', '') or '')
    text = str(row.get('text', '') or '')
    message_text = [
        {"role": "system",
         "content": """
         You are a neutral unbiased expert in world affairs, business and corporate responsibility.
         Your task is to determine whether articles are related to ESG (Environment, Social and Governance) incidents. 
         Rules to follow:
         - A company must be mentioned in the article.
         - The incident is negative, and the company is being criticized or accused of something.
         - The incident must be related to the company's operations, products, or services.
         You output either True or False based on the above rules. True if the document is about an ESG incident, False otherwise.
         """},
        {"role": "user", "content": f"""
        ```
        {title}
        {text}
        ```
        """}
    ]
    try:
        completion = client.chat.completions.create(
            messages=message_text,
            **CHAT_COMPLETION_PARAMS
        )
        response = completion.choices[0].message.content.lower().strip()
        webzio_data.loc[index, 'ai_response'] = 1 if 'true' in response else 0
    except Exception as e:
        print(f"Error processing document: {e}")
        webzio_data.loc[index, 'ai_response'] = 0

webzio_data = webzio_data[webzio_data['ai_response'] == 1]
webzio_data.to_parquet(MODEL_OUTPUT_PATH, index=False)

100%|██████████| 19662/19662 [1:39:24<00:00,  3.30it/s]   


In [225]:
webzio_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1941 entries, 5 to 19648
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uuid                               1941 non-null   object 
 1   url                                1941 non-null   object 
 2   ord_in_thread                      1941 non-null   int64  
 3   parent_url                         0 non-null      object 
 4   author                             1769 non-null   object 
 5   published                          1941 non-null   object 
 6   title                              1941 non-null   object 
 7   text                               1941 non-null   object 
 8   highlightText                      1941 non-null   object 
 9   highlightTitle                     1941 non-null   object 
 10  highlightThreadTitle               1941 non-null   object 
 11  language                           1941 non-null   object 
 

### Entity Extraction

As a first step in our data we extract the entities, as this data could benefit for the pattern extraction

In [226]:
entity_extraction_prompt = """
You are a neutral and unbiased expert in world affairs, business, and corporate responsibility, with specialized knowledge
in Named Entity Recognition (NER).
Your task is to extract and categorize all named entities from a given news article while ensuring accuracy and relevance.

Entity Extraction Guidelines:
Focus on extracting real and named entities only. Ignore generic terms like "a person" or "a company."
Categorize entities into the following types:
- PER (Persons): Individuals mentioned in the article.
- ORG (Organizations & Companies): Businesses, institutions, or recognized groups.
- LOC (Locations): Countries, cities, and geographical places.

If an entity appears in different forms (acronyms, full names, alternative spellings, etc.), group them under a single list.
Example: "United States" and "US" → ["United States", "US"]
Example: "X" and "X.com" → ["X", "X.com"]

Additional Considerations:
Since the input is a news article, explicitly ignore metadata or unrelated elements such as:
- Author names
- News categories, timestamps, and URLs
- Website-related text (e.g., cookie pop-ups)

Example Input & Output:

# Article:
Date: 01/01/2024
Apple Inc. is a company that makes computers. Its most famous one is the MacBook Pro.
Apple's HQ is in Los Angeles; however, the company has been criticized by Amazon for using forced labor in China.
Jeff Bezos, the CEO of Amazon, has been vocal about this issue. MacBooks have been found in Amazon's warehouses.
Meanwhile, Tesla has announced its latest Model S Plaid, which competes with Porsche Taycan.

# Expected Output 
{
    "ORG": [["Apple Inc.", "Apple"], ["Amazon"], ["Tesla"], ["Porsche"]],
    "PER": [["Jeff Bezos"]],
    "LOC": [["Los Angeles"], ["China"]]
}

# Article
2023-12-28 21:15:49
Published 28. December 2023, 10:15 p.m
Seattle, USA: Boeing in trouble again because of a loose screw on a 737 MAX
Boeing is in trouble again with its 737 MAX: After an airline discovered a missing nut, all aircraft of the type are now to be inspected.
1 / 4
An airline discovered a missing nut on a bolt. A loose screw was also discovered on a 737 MAX that had not yet been delivered.
imago images/ZUMA Wire
Now all machines worldwide are to be checked.
REUTERS
The problem concerns a tie rod used to control rudder movements.
REUTERS
That's what it's about
-
An airline discovered a missing nut on a control rod on a Boeing 737 MAX.
-
The US aviation safety authority has now asked all airlines to check the relevant component.
-
The aircraft manufacturer's shares fell after the news.
The US aircraft manufacturer Boeing cannot relax - this time it is about a bolt in the rudder control system of the 737 MAX model.
The Federal Aviation Administration (FAA) urged operators of newer single-aisle aircraft in a statement Thursday to inspect certain control rods used to control
rudder movements for loss of parts. The agency called in a statement to “closely monitor targeted inspections of Boeing 737 MAX aircraft for the possibility of a
loose screw in the rudder control system.”
List of Boeing shows which airlines rely on the type.
Have you been following on Whatsapp for 20 minutes?
Stay informed and subscribe to the 20 Minutes WhatsApp channel: Then you will receive an update with our most moving stories directly to your cell phone in the
morning and evening - hand-picked, informative and inspiring.
(DPA/trx)
1703804980
#Boeing #trouble #due #missing #nut #MAX

# Output
{
    "ORG": [["Boeing"], ["Federal Aviation Administration", "FAA"]],
    "PER": [],
    "LOC": [["Seattle"], ["USA", "US"]]
}
"""

In [227]:
webzio_data['entity_response'] = ''
for index, row in tqdm(webzio_data.iterrows(), total=len(webzio_data)):
    entity_resp = row['entity_response']
    if isinstance(entity_resp, str) and entity_resp.strip() != '' and entity_resp != 'ERROR':
        continue

    message_text = [
        {"role": "system", "content": entity_extraction_prompt},
        {"role": "user", "content": f"""
        # Article
        {row['title'] if row['title'] else ''}
        {row['text'] if row['text'] else ''}

        # Output
        """}
    ]
    try:
        completion = client.chat.completions.create(
            messages=message_text,
            **CHAT_COMPLETION_PARAMS
        )
        webzio_data.at[index, 'entity_response'] = completion.choices[0].message.content
    except Exception as e:
        print(f"Error processing document: {e}")
        webzio_data.at[index, 'entity_response'] = 'ERROR'

def clean_json_text(json_text):
    if not isinstance(json_text, str):
        return {}
    try:
        for tag in ('```json', '```'):
            json_text = json_text.replace(tag, '')
    except Exception as e:
        print(f'Error cleaning json: {e}')
        print(json_text)
    try:
        return json.loads(json_text)
    except Exception as e:
        print(f'Error loading json: {e}')
        print(json_text)
        return {}

webzio_data['json_entities'] = webzio_data['entity_response'].apply(clean_json_text)
webzio_data['json_entities'] = webzio_data['json_entities'].apply(json.dumps)
webzio_data.to_parquet(MODEL_OUTPUT_PATH, index=False)

100%|██████████| 1941/1941 [30:30<00:00,  1.06it/s] 


### Regulation Summarization

Here we pick as our regulation the UNGC Principles.
The text in for the UNGC principle has been manually extracted from https://unglobalcompact.org/what-is-gc/mission/principles

In [228]:
regulation_name = "UN Global Compact (UNGC) Principle"
regulation_dir = "ungc_principles"

regulation_summary_prompt = """
You are an expert on {regulation_name} with comprehensive knowledge of their content, context, and application.

Task: Summarize the following {regulation_name} while maintaining all substantive requirements and actionable guidance.
This summary will serve as the foundation for creating pattern recognition algorithms to identify principles in news articles.

Summary Guidelines:
- Create a concise summary (approximately 300-500 words)
- Maintain all key requirements, obligations, and implementation guidance
- Preserve specific criteria that businesses must satisfy
- Include relevant standards, frameworks, or measurement approaches mentioned
- Omit purely historical context, duplicative statements, or non-essential background

Structure:
1. Begin with a 1-2 sentence overview of the {regulation_name}'s core focus
2. Organize content using clear markdown headings and subheadings
3. Use bullet points for listing specific requirements or actions
4. Bold key terms or thresholds that would be essential for pattern matching

The {regulation_name} to summarize is:
{sub_regulation}

Output only the formatted summary without introduction, explanation, or commentary.
"""

In [210]:
for filename in os.listdir(regulation_dir):
    if filename.endswith('.txt') and not filename.endswith('summary.txt') and filename.rstrip('.txt').split('-')[-1].isdigit():
        with open(os.path.join(regulation_dir, filename), 'r') as f:
            ungc_principle = f.read()
        print(f"Processing {filename}...")
        completion = client.chat.completions.create(
            messages=[{"role": "system",
                       "content": regulation_summary_prompt.format(regulation_name=regulation_name, sub_regulation=ungc_principle)}],
            **CHAT_COMPLETION_PARAMS
            )
        with open(os.path.join(f'{regulation_dir}/{filename[:-4]}-{MODEL_NAME_STR}-summary.txt'), 'w') as f:
            f.write(completion.choices[0].message.content)

Processing principle-8.txt...
Processing principle-5.txt...
Processing principle-7.txt...
Processing principle-3.txt...
Processing principle-6.txt...
Processing principle-10.txt...
Processing principle-4.txt...
Processing principle-1.txt...
Processing principle-2.txt...
Processing principle-9.txt...


### Create Patterns

In [253]:
NUMBER_OF_PATTERNS = 1
NUMBER_OF_REGULATIONS = 10

In [254]:
pattern_prompt = """
You are a neutral and unbiased expert in business ethics, corporate responsibility, and international standards,
with specialized knowledge in {regulation_name} and their practical application.

Your task is to develop an extraction framework that can systematically identify potential {regulation_name} violations in news articles.
This framework will:
1. Define precise extraction patterns calibrated to each {regulation_name}
2. Extract structured information from news text using these patterns
3. Match extracted information to specific {regulation_name} and categorize potential violations

## Pattern Requirements:
- Each pattern must be formalized as a relation triple (Entity A, Action, Entity B) where:
  * Entity A is typically the business/corporation or its representatives
  * Action is the specific behavior, decision, or practice that may violate {regulation_name}
  * Entity B is the affected party, resource, or context

- Patterns must capture both:
  * Direct violations (explicit actions that contradict principles)
  * Indirect violations (failures to implement required safeguards or due diligence)

- Patterns should account for varied linguistic expressions of the same underlying violation

## {regulation_name}:
{concatenated_regulation}

## Examples of Effective Extraction Patterns:

1. Human Rights Violation Pattern: (Company, exploits/endangers/violates, rights of community/workers/individuals)
2. Environmental Harm Pattern: (Company, pollutes/damages/depletes, ecosystem/resource/environment)
3. Labor Abuse Pattern: (Company, restricts/prevents/penalizes, worker organization/collective action)
4. Corruption Pattern: (Company, bribes/pays/transfers value to, government official/regulator/decision-maker)

## Your Deliverables:
1. Create {number_of_patterns} specialized extraction patterns for EACH {regulation_name} ({total_number_of_patterns} total patterns)
2. For each pattern, provide:
   - Formal triple structure (A, B, C)
   - Add an fictitious example of a sentence that could be extracted with said pattern (in its triple structure) in a field called "To look for"
   - Add also a counter example of a sentence that should not be extracted with said pattern (in its triple structure) in a field called "To ignore"
3. Number the patterns sequentially from 1 to 30 (Do not number them like 1.x, 2.x, etc. Just use 1, 2, 3, ..., {total_number_of_patterns})
"""

parameters = {
    "concatenated_regulation": '',
    "regulation_name": regulation_name,
    "number_of_patterns": NUMBER_OF_PATTERNS,
    "total_number_of_patterns": NUMBER_OF_PATTERNS * NUMBER_OF_REGULATIONS,
}
def extract_principle_number(filename):
    match = re.search(r'(\d+)-{}-summary.txt$'.format(MODEL_NAME_STR), filename)
    return int(match.group(1)) if match else float('inf')

all_sub_regulations = []

# Get all summary filenames and sort them by principle number
summary_filenames = [f for f in os.listdir(regulation_dir) if f.endswith(f'{MODEL_NAME_STR}-summary.txt')]
summary_filenames_sorted = sorted(summary_filenames, key=extract_principle_number)


for filename in summary_filenames_sorted:
    with open(os.path.join(regulation_dir, filename), 'r') as f:
        sub_regulation = f.read()
    all_sub_regulations.append(sub_regulation)

parameters["concatenated_regulation"] = "\n\n".join(all_sub_regulations)

message_text = [
    {"role": "system",
        "content": f"""
        {pattern_prompt.format(**parameters)}
        """
    }
]
completion = client.chat.completions.create(
    messages=message_text,
   **CHAT_COMPLETION_PARAMS
   )
if not os.path.exists('patterns'):
    os.makedirs('patterns')
with open(f'patterns/extraction_patterns_{MODEL_NAME_STR}_{NUMBER_OF_PATTERNS}.txt', 'w') as f:
    f.write(completion.choices[0].message.content)

### Predict Generated Patterns

In [258]:
query_regulation_patterns = """
# Task: Extract {regulation_name} Violations from News Articles

You are a strict pattern matcher. 
You must only match text if it exactly or strongly resembles one of the known patterns.
Do not infer or assume meaning that is not explicitly present in the text.

Extract patterns from news articles that indicate potential violations of {regulation_name}. 
The goal is to identify as many valid patterns as possible that show how businesses may be violating these important international standards.

## Important Instructions:
- Extract information that matches one of the predefined patterns
- If a company or entity name is missing but clearly implied, include it
- Output only in the format specified below
- Keep extracted patterns concise without losing critical meaning
- If no patterns match, return an empty `"patterns"` array

"""

expected_format = """
## Expected JSON Output Format
{
    "patterns": [
        {
            "pattern1": [["Company A", "violates", "human rights of indigenous communities"]]
        },
        {
            "pattern20": [["Mining Corporation", "engages in", "environmentally harmful practices"]]
        },
        {
            "pattern28": [
                ["Telecom Company", "engages in", "bribery of local officials"],
                ["Same Company", "engages in", "bribery of regulatory officials"]
            ]
        }
    ]
}

## Final Notes:
- If multiple instances of a pattern exist, include all in the respective array
- If a pattern isn't found in the text, don't include it in the output
- Ensure correct structure with no trailing commas, only use double quotes
- Prioritize accuracy over quantity of extractions
- You must only match text if it exactly or strongly resembles one of the known patterns.
- Do not infer or assume meaning that is not explicitly present in the text.
- Do not output anything except the matched patterns in the specified format.

# Follow these 3 steps : 
1. Extract potential triples from the text.
2. Match those triples to predefined patterns.
3. Check the triples are exact or strong matches. 
4. Output only the matched patterns in the specified format.
"""

In [256]:
webzio_data = pd.read_parquet(MODEL_OUTPUT_PATH)

In [257]:
webzio_data['pattern_response'] = ''

In [259]:
with open(os.path.join('patterns', f'extraction_patterns_{MODEL_NAME_STR}_{NUMBER_OF_PATTERNS}.txt'), 'r') as f:
    generated_patterns = f.read()

for index, row in tqdm(webzio_data.iterrows(), total=len(webzio_data)):
    if row['pattern_response'] != '' and row['pattern_response'] != 'ERROR' and type(row['pattern_response']) is str:
        continue
    message_text = [
        {"role": "system",
         "content": query_regulation_patterns.format(regulation_name=regulation_name) + generated_patterns + expected_format},
        {"role": "user", "content": f"""
        # Article
        {row['title'] if row['title'] else ''}
        {row['text'] if row['text'] else ''}

        # Entities
        {row['json_entities'] if row['json_entities'] else ''}
        
        # Output
        """}
    ]
    try:
        completion = client.chat.completions.create(
            messages=message_text,
            **CHAT_COMPLETION_PARAMS
            )
        webzio_data.at[index, 'pattern_response'] = completion.choices[0].message.content
    except Exception as e:
        print(f"Error processing document: {e}")
        webzio_data.at[index, 'pattern_response'] = 'ERROR'

100%|██████████| 1941/1941 [20:42<00:00,  1.56it/s] 


In [260]:
webzio_data['ungc_patterns'] = webzio_data['pattern_response'].apply(clean_json_text)
webzio_data['ungc_patterns'] = webzio_data['ungc_patterns'].apply(json.dumps)
webzio_data.to_parquet(base_folder / f"research_paper_data_{MODEL_NAME_STR}_{NUMBER_OF_PATTERNS}.parquet", index=False)

Error loading json: Expecting property name enclosed in double quotes: line 8 column 13 (char 229)
{
    "patterns": [
        {
            "pattern7": [["Apple", "neglects", "environmental precautions"]]
        },
        {
            "pattern8": [["Apple", "fails to promote", "environmental responsibility"]],
            ["Samsung", "fails to promote", "environmental responsibility"],
            ["Google", "fails to promote", "environmental responsibility"]
        }
    ]
}
Error loading json: Expecting property name enclosed in double quotes: line 8 column 13 (char 229)
{
    "patterns": [
        {
            "pattern7": [["Apple", "neglects", "environmental precautions"]]
        },
        {
            "pattern8": [["Apple", "fails to promote", "environmental responsibility"]],
            ["Samsung", "fails to promote", "environmental responsibility"],
            ["Google", "fails to promote", "environmental responsibility"]
        }
    ]
}
Error loading json: Expectin

### Double checking predictions

In [143]:
double_check_prompt = """
You are given a text and a list of patterns. Each pattern describes a type of relationship or action involving specific entities.
Your task is to examine the text and return a list of triples. 
The triple should consist of a pattern ID, one sentence from the text that correspond to the pattern, and an explanation why the sentence supports the pattern.

Only include a pattern if the content of the text clearly supports or describes it. Do not infer beyond what is explicitly or strongly implied.
If you are not sure about a pattern, do not include it.
If the text does not support the pattern and instead contradicts it, do not include it.

If there are multiple times the same pattern, only include the first one.

### TEXT:
{article_text}

### PATTERNS:
{patterns}

### OUTPUT FORMAT:
[("pattern1", <sentence in the text>, <explanation why the sentence supports the pattern>), ("pattern4", ...)]
"""

In [ ]:
webzio_data['double_check_response'] = ''

for index, row in tqdm(webzio_data.iterrows(), total=len(webzio_data)):
    if row['double_check_response'] != '' and row['double_check_response'] != 'ERROR' and type(row['double_check_response']) is str:
        continue
    message_text = [
        {"role": "user",
         "content": double_check_prompt.format(
            article_text=row['text'] if row['text'] else '',
            patterns=row['ungc_patterns'] if row['ungc_patterns'] else '')},
    ]
    try:
        completion = client.chat.completions.create(
            messages=message_text,
            **CHAT_COMPLETION_PARAMS
            )
        webzio_data.at[index, 'double_check_response'] = completion.choices[0].message.content
    except Exception as e:
        print(f"Error processing document: {e}")
        webzio_data.at[index, 'double_check_response'] = 'ERROR'

webzio_data.to_parquet(MODEL_OUTPUT_PATH, index=False)

100%|██████████| 1491/1491 [11:47<00:00,  2.11it/s] 


# Compute Metrics

In [261]:
prediction_data = pd.read_parquet(base_folder / f"research_paper_data_{MODEL_NAME_STR}_{NUMBER_OF_PATTERNS}.parquet")
analyzed_data = pd.read_csv('./labeled_data/ungc_label_200.csv')
merged_data = pd.merge(analyzed_data, prediction_data, on='uuid', how='left')

## Predicted Labels

In [263]:
def convert_predicted_unchecked_labels_to_ungc_principles(label_list):
    ungc_labels = set()
    try:
        if isinstance(label_list, float):
            return []
        for label in ast.literal_eval(label_list)['patterns']:
            pattern_id = list(label.keys())[0].strip('pattern')
            ungc_labels.add(math.ceil(int(pattern_id) / NUMBER_OF_PATTERNS))
    except (ValueError, SyntaxError):
        numbers = re.findall(r'pattern(\d+)', label_list)
        for num in numbers:
            ungc_labels.add(math.ceil(int(num) / NUMBER_OF_PATTERNS))
    return list(ungc_labels)

merged_data['predicted_labels'] = merged_data['pattern_response'].apply(convert_predicted_unchecked_labels_to_ungc_principles)

In [264]:
def convert_human_label_to_list(human_label):
    human_label_set = []
    for el in human_label.strip('[').strip(']').split(' '):
        if el.strip() != '':
            human_label_set.append(int(el.strip()))
    return human_label_set

merged_data['labels'] = merged_data['human_label'].apply(convert_human_label_to_list)

## Double Checked Labels

In [ ]:
import math 
import ast
import re

def convert_predicted_labels_to_ungc_principles(label_list):
    ungc_labels = set()
    if not isinstance(label_list, str) or label_list.strip() == '':
        return []
    try:
        for label in ast.literal_eval(label_list):
            pattern_id = label[0].strip('pattern')
            ungc_labels.add(math.ceil(int(pattern_id) / NUMBER_OF_PATTERNS))
    except (ValueError, SyntaxError):
        numbers = re.findall(r'pattern(\d+)', label_list)
        for num in numbers:
            ungc_labels.add(math.ceil(int(num) / NUMBER_OF_PATTERNS))
    return list(ungc_labels)

merged_data['predicted_checked_labels'] = merged_data['double_check_response'].apply(convert_predicted_labels_to_ungc_principles)

## Metrics Table

In [265]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(merged_data['labels'])
y_pred_bin = mlb.transform(merged_data['predicted_labels'])

labels = mlb.classes_
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Start LaTeX output
latex_lines = []

for i, label in enumerate(labels):
    tn, fp, fn, tp = conf_matrices[i].ravel()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else float('nan')
    accuracy  = (tp + tn) / (tp + tn + fp + fn)

    line = f"{label} & {tp} & {fn} & {fp} & {tn} & {precision:.2f} & {accuracy:.2f} & {recall:.2f} & {f1:.2f} \\\\"
    latex_lines.append(line)

# Join and print LaTeX
latex_output = "\n".join(latex_lines)
print(latex_output)


1 & 39 & 27 & 28 & 106 & 0.58 & 0.72 & 0.59 & 0.59 \\
2 & 21 & 5 & 18 & 156 & 0.54 & 0.89 & 0.81 & 0.65 \\
3 & 16 & 2 & 69 & 113 & 0.19 & 0.65 & 0.89 & 0.31 \\
4 & 7 & 7 & 38 & 148 & 0.16 & 0.78 & 0.50 & 0.24 \\
5 & 5 & 2 & 31 & 162 & 0.14 & 0.83 & 0.71 & 0.23 \\
6 & 9 & 0 & 48 & 143 & 0.16 & 0.76 & 1.00 & 0.27 \\
7 & 17 & 4 & 52 & 127 & 0.25 & 0.72 & 0.81 & 0.38 \\
8 & 15 & 3 & 35 & 147 & 0.30 & 0.81 & 0.83 & 0.44 \\
9 & 0 & 1 & 0 & 199 & 0.00 & 0.99 & 0.00 & nan \\
10 & 11 & 5 & 40 & 144 & 0.22 & 0.78 & 0.69 & 0.33 \\


For a more visually appealing table you can copy-paste the numbers below

In [266]:
from IPython.display import display, Latex

latex_code = r"""
$$
\begin{array}{lcccccccc}
\textbf{Principle} & \textbf{TP} & \textbf{FN} & \textbf{FP} & \textbf{TN} & \textbf{Precision} & \textbf{Accuracy} & \textbf{Recall} & \textbf{F1} \\
\hline
1 & 39 & 27 & 28 & 106 & 0.58 & 0.72 & 0.59 & 0.59 \\
2 & 21 & 5 & 18 & 156 & 0.54 & 0.89 & 0.81 & 0.65 \\
3 & 16 & 2 & 69 & 113 & 0.19 & 0.65 & 0.89 & 0.31 \\
4 & 7 & 7 & 38 & 148 & 0.16 & 0.78 & 0.50 & 0.24 \\
5 & 5 & 2 & 31 & 162 & 0.14 & 0.83 & 0.71 & 0.23 \\
6 & 9 & 0 & 48 & 143 & 0.16 & 0.76 & 1.00 & 0.27 \\
7 & 17 & 4 & 52 & 127 & 0.25 & 0.72 & 0.81 & 0.38 \\
8 & 15 & 3 & 35 & 147 & 0.30 & 0.81 & 0.83 & 0.44 \\
9 & 0 & 1 & 0 & 199 & 0.00 & 0.99 & 0.00 & nan \\
10 & 11 & 5 & 40 & 144 & 0.22 & 0.78 & 0.69 & 0.33 \\
\end{array}
$$
"""
display(Latex(latex_code))


<IPython.core.display.Latex object>